In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%pip install mglearn
import mglearn

In [ ]:
#Open JobApplicants.csv file
df = pd.read_csv('Train.csv')

In [ ]:
df.head()

,Unnamed: 0,w,Age,Accessibility,EdLevel,Employment,Gender,MentalHealth,MainBranch,YearsCode,YearsCodePro,Country,PreviousSalary,HaveWorkedWith,ComputerSkills,Employed
0,35381,35381,>35,Yes,Other,1,Man,Yes,NotDev,35,15,United Kingdom of Great Britain and Northern I...,77556.0,C;C++;Go;PowerShell;Python;Ansible;Chef;Docker...,16,0
1,1453,1453,>35,No,PhD,0,Man,No,Dev,35,30,United Kingdom of Great Britain and Northern I...,124092.0,C#;HTML/CSS;LISP;ASP.NET Core ;jQuery;Microsof...,7,1
2,24572,24572,<35,No,Master,1,NonBinary,Yes,Dev,18,2,Denmark,78672.0,Bash/Shell;HTML/CSS;Java;JavaScript;Python;SQL...,12,1
3,42014,42014,<35,No,Undergraduate,1,Man,Yes,Dev,10,1,France,44790.0,Rust;Docker;Kubernetes;AWS;Google Cloud;MongoD...,7,0
4,26711,26711,>35,No,Master,0,Man,No,Dev,21,17,United Kingdom of Great Britain and Northern I...,155112.0,C#;HTML/CSS;JavaScript;Node.js;SQL;TypeScript;...,16,1


In [ ]:
df.drop(['MentalHealth'], axis=1, inplace=True)

In [ ]:
#What correlates the most with "employed" column?
df.corr()['Employed'].sort_values()

C:\Users\Simon\AppData\Local\Temp\ipykernel_9012\1276420888.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()['Employed'].sort_values()


Employment       -0.010255
YearsCode        -0.008982
YearsCodePro     -0.001812
PreviousSalary    0.002171
Unnamed: 0        0.071393
w                 0.071393
ComputerSkills    0.586196
Employed          1.000000
Name: Employed, dtype: float64

In [ ]:
# Identify categorical and numerical columns
categorical_cols = ['Age', 'EdLevel', 'Gender', 'MainBranch']
numerical_cols = ['YearsCode', 'YearsCodePro', 'PreviousSalary', 'ComputerSkills']
#Why was it so weird if I use the under written code???
#numerical_cols.extend(['WW ' + language for language in unique_languages])
#Take WW out of array because it does not contain anything
#numerical_cols.remove('WW ')
#print(numerical_cols)


# Defining the feature matrix X by selecting all columns except Employed 
X = lr[categorical_cols + numerical_cols]  
#X = df.drop(columns=['Employed'])# axis=1 refers to columns
# Defining the target variable y by selecting only the 'Employed' column
y = lr['Employed']

# Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score 

In [ ]:
# Handle Categorical Variables with One-Hot Encoding
rf = pd.get_dummies(df, columns=["Age", "Accessibility", "EdLevel", "Gender", "Country"], drop_first=True)

# Remove non-numeric columns or columns that are not relevant for modeling
# Faktummet at vi fjerner computerskills er også godt da det er en aggregat 
rf = rf.drop(columns=["MainBranch", "ComputerSkills"])

In [ ]:
# Assuming "Employed" is the target variable, and the other columns are features
X = rf.drop("Employed", axis=1)  # Features
y = rf["Employed"]  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create a Random Forest model (for classification)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=3)

# Fit the model on the training data
rf_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = rf_model.predict(X_test)

# Evaluate the model (for classification, you can use metrics like accuracy)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

Accuracy: 0.853577748536685
F1 Score: 0.8687197428908506
Precision: 0.8348580811634999
Recall: 0.905444369063772


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define the hyperparameters and their possible values
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create a GridSearchCV object
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='f1')

# Fit the grid search to your data
grid_search.fit(X_train, y_train)

# Get the best parameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
